In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore") 

In [2]:
def GetSubFiles(dir, ExtensionList = []):
    print("=" * 80, "- Begin: GetSubFiles")

    "Get a list of immediate subfiles"
    all_names = next(os.walk(dir))[2]
    subfile_names = []
    if(len(ExtensionList) > 0):
        for filename in all_names:
            fname, fextension = os.path.splitext(filename)
            if any(ext in fextension for ext in ExtensionList):
                subfile_names.append(dir + "/" + filename)
    else:
        subfile_names = [dir + "/" + x for x in all_names]
    
    print(f"There are {len(subfile_names)} files are found")
    subfile_names.sort()
    print("Here is some samples :")
    [print(x) for x in subfile_names[0: min(len(subfile_names), 5)]]

    print("=" * 80, "- End: GetSubFiles")
    return subfile_names

In [3]:
data_path = "/Users/mac/Documents/Computer Vision/CPV project"

In [4]:
os.chdir(data_path)

In [5]:
len(os.listdir())

5

In [10]:
DatasetFiles = GetSubFiles("Images/train/0", ExtensionList = ["jpg"])
print(DatasetFiles)

================================================================================ - Begin: GetSubFiles
There are 4 files are found
Here is some samples :
Images/train/0/0.jpg
Images/train/0/1.jpg
Images/train/0/2.jpg
Images/train/0/3.jpg
================================================================================ - End: GetSubFiles
['Images/train/0/0.jpg', 'Images/train/0/1.jpg', 'Images/train/0/2.jpg', 'Images/train/0/3.jpg']


In [11]:
root = "Images/train"
# lặp qua thư mục con (os.walk)
categories = [x[0] for x in os.walk(root)]
len(categories)

784

In [12]:
from PIL import Image
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers
from tensorflow.keras.models import load_model, model_from_json
import tensorflow as tf
import PIL

In [13]:
batch_size = 32
img_height = 224
img_width = 224

In [14]:
root = "Images/"

# tạo data
train_datagen = ImageDataGenerator(
    shear_range=10, # kéo dãn image
    zoom_range=0.2,
    horizontal_flip=True, # lật ngang
    # tiền xữ lý theo thuật toán đầu vào
    preprocessing_function=preprocess_input
)

train_dataset = train_datagen.flow_from_directory(
    directory=root + "train",
    batch_size=batch_size,
    shuffle=True,
    target_size = (img_width, img_height),
    # kết quả đầu ra sẽ là vector xác suất
    class_mode= "categorical"
)


valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

valid_dataset = valid_datagen.flow_from_directory(
    directory=root + "validation",
    batch_size=batch_size,
    shuffle=True,
    target_size = (img_width, img_height),
    # kết quả đầu ra sẽ là vector xác suất
    class_mode= "categorical"
)

Found 3752 images belonging to 783 classes.
Found 2355 images belonging to 541 classes.


In [15]:
class_dict = train_dataset.class_indices
class_name = list(class_dict.keys())
print(class_name)

['0', '1', '10', '1000', '1001', '1002', '1006', '1007', '101', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '102', '103', '104', '105', '106', '107', '109', '110', '111', '112', '113', '115', '116', '117', '118', '119', '12', '120', '122', '124', '125', '126', '127', '128', '13', '132', '133', '134', '135', '136', '138', '139', '140', '141', '142', '143', '146', '148', '149', '150', '151', '154', '155', '156', '157', '158', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '187', '188', '189', '19', '191', '192', '193', '194', '195', '196', '197', '198', '199', '20', '200', '201', '202', '203', '205', '206', '207', '208', '209', '21', '210', '211', '214', '215', '217', '219', '220', '223', '224', '226', '227', '228', '23', '230', '231', '233', '234', '236', '237', '238', '240', '241', '242', '243', '244', '245', '246', '248', '249', '25', 

In [16]:
# built resnet
conv_base = ResNet50(
    include_top=False, # chỉ lấy phía trước vì dùng kiến trúc transfer learning
    weights= "imagenet"
)

for layer in conv_base.layers:
    # không train lại weight của 50 layer trên
    layer.trainable = False

Metal device set to: Apple M1


2023-06-28 12:01:41.583750: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-28 12:01:41.583942: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
x = conv_base.output
# Đi qua mạng inferance
x = layers.GlobalAveragePooling2D()(x)
# x trên có dense = 2048
x = layers.Dense(128, activation="relu")(x)

# trả về xác suất của 4 object
predictions = layers.Dense(len(class_name), activation="softmax")(x)
# vì tổng xác suất trên sẽ không bằng 1 nên cần phải đưa về probability-space (softmax) sẽ có tổng bằng 1

# define model
# backbone
model = Model(conv_base.input, predictions)

In [18]:
# categorical_crossentropy: -ylogy(p)   (xác suất có thuộc lớp đó hay không)
model.compile(loss="categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [20]:
# train
history = model.fit(train_dataset, 
                    steps_per_epoch=train_dataset.samples // batch_size,
                    epochs=20,
                    validation_data=valid_dataset,
                    validation_steps=valid_dataset.samples // batch_size
                   )

Epoch 1/20


2023-06-28 12:01:52.576488: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-28 12:01:53.772279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


117/117 [==============================] - ETA: 0s - loss: 6.4056 - accuracy: 0.0210

2023-06-28 12:02:35.376349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/Users/mac/miniconda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/mac/miniconda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/mac/miniconda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/mac/miniconda/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/mac/miniconda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/mac/miniconda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/Users/mac/miniconda/lib/p